# Garden Tutorial Finale: Execute Your Model Remotely

This notebook is part of [the Garden AI tutorial](https://garden-ai.readthedocs.io/en/latest/user_guide/tutorial/).
This notebook will show you how to invoke your iris classification model now that you've published it.

### Install the Garden SDK

In [2]:
# We'll need to install Garden in this Colab session
!pip install garden-ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 788.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.4/257.4 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

## Create a GardenClient and Log in with Globus

We need a GardenClient to run entrypoint functions. You'll have to log in like you did on the command line. Run the cell, click through the link, and paste the authorization code in the text box. Then you'll have a working GardenClient.

In [3]:
from garden_ai import GardenClient
garden_client = GardenClient()

Authenticating with Globus in your default web browser:

Please enter the code here :

Please authenticate with Globus here:
------------------------------------
https://auth.globus.org/v2/oauth2/authorize?client_id=4cf29807-cf21-49ec-9443-ff9a3fb9f81c&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=https%3A%2F%2Fauth.globus.org%2Fscopes%2Ffacd7ccc-c5f4-42aa-916b-a0e270e2c2a9%2Fall+openid+urn%3Aglobus%3Aauth%3Ascope%3Aauth.globus.org%3Amanage_projects&state=_default&response_type=code&code_challenge=EfDO2lGvclwEZuY4JKW-q7vuVnBx2dMm3VKqrD2r1pI&code_challenge_method=S256&access_type=offline&prefill_named_grant=9ccd7183789e&prompt=login
------------------------------------

Enter the resulting Authorization Code here: LKFN292W82oE0m1nDG1KUWr4RQQckZ


### Fetch Your Published Garden

Now you will pull down a copy of your garden from the Garden backend. If you don't remember your garden's doi, check with `garden-ai garden list` in a terminal on your computer.

In [4]:
your_garden_doi = '10.26311/ep98-br79' # Paste in your garden's DOI here!
garden = garden_client.get_published_garden(your_garden_doi)
garden

Title,Authors,Doi
Bandgap model,['Ryan Jacobs'],10.26311/3p8f-se33
Concrete model,['Ryan Jacobs'],10.26311/fq14-0d14
Piezoelectric displacement model,['Ryan Jacobs'],10.26311/ne10-cw51
Dielectric constant model,['Ryan Jacobs'],10.26311/xc43-an24
Dilute solute diffusion model,['Ryan Jacobs'],10.26311/da1c-y367
Double perovskite bandgap model,['Ryan Jacobs'],10.26311/1bgd-3w38
Elastic tensor model,['Ryan Jacobs'],10.26311/hh7f-ar11
Exfoliation energy model,['Ryan Jacobs'],10.26311/byhv-bh08
High entropy hardness model,['Ryan Jacobs'],10.26311/e9rx-7020
Heusler magnetization model,['Ryan Jacobs'],10.26311/9fan-dk80


### Execute Your Model

The entrypoint function from the notebook you used to publish will be a method on the garden it was published to. We just need to pass it the same input we passed it locally and call it. Garden will serialize your input, send it to a worker on the Garden demo computing endpoint, and send the response back to the notebook.

#### Entrypoint Cold Start

Your entrypoint might take a minute or so to run the first time you execute it. That's because the remote computing endpoint has to download the underlying container. But after that your entrypoint will be warmed up and only take a few seconds.

In [5]:
# Predict bandgaps

comps = ['Hg0.7Cd0.3Te', 'CuBr', 'LuP']
result = garden.predict_bandgap(comps)
result

Output()

(worker SDK version: 2.16.0; worker OS: Linux-3.10.0-1160.90.1.el7.x86_64-x86_64-with-glibc2.36)


,Compositions,Predicted bandgap (eV),Ebar bandgap (eV),rmse/std_y Domain Prediction from Max F1,cdf_area Domain Prediction from Max F1
0,Hg0.7Cd0.3Te,0.296223,0.127444,ID,ID
1,CuBr,3.000977,0.119141,ID,ID
2,LuP,1.049378,0.461603,ID,ID


In [6]:
# Predict concrete compressive strength
import pandas as pd
d_test = {'Cement (component 1)(kg in a m^3 mixture)' : [540, 540],
              'Blast Furnace Slag (component 2)(kg in a m^3 mixture)': [0, 0],
              'Fly Ash (component 3)(kg in a m^3 mixture)': [0, 0],
              'Water  (component 4)(kg in a m^3 mixture)': [162, 162],
              'Superplasticizer (component 5)(kg in a m^3 mixture)': [2.5, 2.5],
              'Coarse Aggregate  (component 6)(kg in a m^3 mixture)': [1040, 1055],
              'Fine Aggregate (component 7)(kg in a m^3 mixture)': [676, 676],
              'Age (day)': [28, 28]}
result = garden.predict_concrete(pd.DataFrame(d_test))
result

Output()

(worker SDK version: 2.16.0; worker OS: Linux-3.10.0-1160.90.1.el7.x86_64-x86_64-with-glibc2.36)


,Predicted concrete compressive strength (MPa),Ebar concrete compressive strength (MPa),d_pred,rmse/std_y Domain Prediction from Max F1,cdf_area Domain Prediction from Max F1,Cement (component 1)(kg in a m^3 mixture),Age (day),Water (component 4)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture)
0,72.5402,8.006114,1.0,ID,OD,540,28,162,0,2.5,676,1040,0
1,65.3002,7.058707,1.0,ID,OD,540,28,162,0,2.5,676,1055,0


In [7]:
# Predict piezoelectric max displacement

comps = ['AgI', 'CdTe', 'BN']
result = garden.predict_piezoelectric(comps)

result

Output()

(worker SDK version: 2.16.0; worker OS: Linux-3.10.0-1160.90.1.el7.x86_64-x86_64-with-glibc2.36)


,Compositions,Predicted log max piezoelectric displacement (C/m2),Ebar log max piezoelectric displacement (C/m2),d_pred,rmse/std_y Domain Prediction from Max F1,cdf_area Domain Prediction from Max F1
0,AgI,-0.914367,0.460204,0.989896,ID,ID
1,CdTe,-1.972526,0.555926,0.958085,ID,ID
2,BN,-0.232440,0.458609,0.979081,ID,ID


In [8]:
# Predict dielectric constant (log scale)

comps = ['Rb2Te', 'CdCl2', 'MnI2']
result = garden.predict_dielectric(comps)

result

Output()

(worker SDK version: 2.16.0; worker OS: Linux-3.10.0-1160.90.1.el7.x86_64-x86_64-with-glibc2.36)


,Compositions,Predicted dielectric constant (log scale),Ebar dielectric constant (log scale),d_pred,rmse/std_y Domain Prediction from Max F1,cdf_area Domain Prediction from Max F1
0,Rb2Te,0.822929,0.096528,0.998561,OD,OD
1,CdCl2,0.843257,0.075977,0.725923,ID,OD
2,MnI2,0.983469,0.134695,0.901362,ID,OD


In [9]:
# Predict dilute solute diffusion barrier

comps = ['AgAg', 'AgCo', 'AgCr']
result = garden.predict_diffusion(comps)

result

Output()

(worker SDK version: 2.16.0; worker OS: Linux-3.10.0-1160.92.1.el7.x86_64-x86_64-with-glibc2.36)


,Compositions,Predicted diffusion barrier (eV),Ebar diffusion barrier (eV),d_pred,rmse/std_y Domain Prediction from Max F1,cdf_area Domain Prediction from Max F1
0,AgAg,1.741324,0.162277,0.687747,ID,OD
1,AgCo,1.857842,0.197090,0.240499,ID,ID
2,AgCr,2.101624,0.151435,0.357934,ID,ID


In [10]:
# Predict double perovskite bandgaps

comps = ['AgNbLaAlO6', 'AgNbLaGaO6', 'AgNbLaInO6']
result = garden.predict_double_perovskite_gap(comps)

result

Output()

(worker SDK version: 2.16.0; worker OS: Linux-3.10.0-1160.90.1.el7.x86_64-x86_64-with-glibc2.36)


,Compositions,Predicted double perovskite bandgap (eV),Ebar double perovskite bandgap (eV),d_pred,rmse/std_y Domain Prediction from Max F1,cdf_area Domain Prediction from Max F1
0,AgNbLaAlO6,4.125667,0.437532,0.812345,ID,OD
1,AgNbLaGaO6,4.309278,0.442133,0.872212,ID,OD
2,AgNbLaInO6,4.222878,0.373651,0.870127,ID,OD


In [11]:
# Predict bulk modulus

comps = ['Nb4CoSi', 'Al(CoSi)2', 'SiOs']
result = garden.predict_elastic_tensor(comps)

result

Output()

(worker SDK version: 2.16.0; worker OS: Linux-3.10.0-1160.90.1.el7.x86_64-x86_64-with-glibc2.36)


,Compositions,Predicted bulk modulus (GPa),Ebar bulk modulus (GPa),d_pred,rmse/std_y Domain Prediction from Max F1,cdf_area Domain Prediction from Max F1
0,Nb4CoSi,194.276091,11.104032,0.656120,ID,OD
1,Al(CoSi)2,173.097596,14.421685,0.164386,ID,OD
2,SiOs,274.151497,29.673087,1.000000,OD,OD


In [12]:
# Predict exfoliation energy

comps = ['HfSiTe', 'As', 'TiOBr']
result = garden.predict_exfoliationE(comps)

result

Output()

(worker SDK version: 2.16.0; worker OS: Linux-3.10.0-1160.92.1.el7.x86_64-x86_64-with-glibc2.36)


,Compositions,Predicted exfoliation energy (eV/atom),Ebar exfoliation energy (eV/atom),d_pred,rmse/std_y Domain Prediction from Max F1,cdf_area Domain Prediction from Max F1
0,HfSiTe,71.815816,40.418549,0.745994,ID,ID
1,As,166.304814,180.582984,0.999654,OD,ID
2,TiOBr,46.774900,69.619055,0.739510,ID,ID


In [13]:
# Predict high entropy alloy hardness

comps = ['Co22.2Cr22.2Fe22.2Ni22.2Mo11.1', 'Al10Co20Cr20Fe20Ni20Mo10', 'Al20Co10Cr20Fe20Ni20Mo10']
result = garden.predict_hea_hardness(comps)

result

Output()

(worker SDK version: 2.16.0; worker OS: Linux-3.10.0-1160.90.1.el7.x86_64-x86_64-with-glibc2.36)


,Compositions,Predicted HEA hardness (HV),Ebar HEA hardness (HV),d_pred,rmse/std_y Domain Prediction from Max F1,cdf_area Domain Prediction from Max F1
0,Co22.2Cr22.2Fe22.2Ni22.2Mo11.1,305.379867,144.593831,1.000000,OD,OD
1,Al10Co20Cr20Fe20Ni20Mo10,485.845467,109.651044,0.922615,ID,OD
2,Al20Co10Cr20Fe20Ni20Mo10,766.510467,58.967527,0.747353,ID,OD


In [14]:
# Predict heusler magnetization

comps_list = ['V2ScAl', 'V2ScGa', 'V2ScIn']
numelec_list = [16, 16, 16]
heuslertype_list = ['Full Heusler', 'Full Heusler', 'Full Heusler']
result = garden.predict_heusler(comps_list, numelec_list, heuslertype_list)

result

Output()

(worker SDK version: 2.16.0; worker OS: Linux-3.10.0-1160.92.1.el7.x86_64-x86_64-with-glibc2.36)


,Predicted magnetization (emu/cm3),Ebar magnetization (emu/cm3),d_pred,rmse/std_y Domain Prediction from Max F1,cdf_area Domain Prediction from Max F1,BCCmagmom_arithmetic_average,NdUnfilled_arithmetic_average,num_electron,NdUnfilled_composition_average,heusler type_0,...,GSenergy_pa_min_value,ElectronAffinity_composition_average,n_ws^third_composition_average,Electronegativity_composition_average,valence_composition_average,ICSDVolume_min_value,GSenergy_pa_composition_average,BCCenergy_pa_composition_average,Polarizability_composition_average,phi_difference
0,28.59694,66.417110,1.0,ID,OD,0.0,5.333333,16,5.75,1,...,-8.939261,40.775,1.485,1.5575,4.0,13.8,-6.977364,-6.925200,12.3500,1.0
1,40.83208,59.145079,1.0,ID,OD,0.0,5.333333,16,5.75,1,...,-8.939261,36.775,1.465,1.6075,4.0,13.8,-6.794072,-6.757251,12.6800,1.0
2,46.02794,70.624555,1.0,ID,OD,0.0,5.333333,16,5.75,1,...,-8.939261,36.775,1.430,1.6000,4.0,13.8,-6.707709,-6.676129,13.0625,1.0


In [15]:
# Predict Li conductivity in solid state electrolytes

comps = ['Li2OHBr', 'Li2.4OH0.6Cl', 'Li2.1OH0.9Cl']
result = garden.predict_Li_conductivity(comps)

result

Output()

(worker SDK version: 2.16.0; worker OS: Linux-3.10.0-1160.90.1.el7.x86_64-x86_64-with-glibc2.36)


,Predicted log Li conductivity (S/cm),Ebar log Li conductivity (S/cm),d_pred,rmse/std_y Domain Prediction from Max F1,cdf_area Domain Prediction from Max F1
0,-6.504979,1.012509,1.0,OD,OD
1,-7.040034,1.113025,1.0,OD,OD
2,-7.119530,0.658016,1.0,OD,OD


In [16]:
# Predict Dmax of metallic glasses

comps = ['Ag1Al19.8Co19.8Cu4.95Zr54.45', 'Ag1Al9.9Cu22.77Fe4.95Zr61.38', 'Ag1Cu5P5Pd79Si10']
result = garden.predict_metallicglass_Dmax(comps)

result


Output()

(worker SDK version: 2.16.0; worker OS: Linux-3.10.0-1160.90.1.el7.x86_64-x86_64-with-glibc2.36)


,Compositions,Predicted Dmax (mm),Ebar Dmax (mm),d_pred,rmse/std_y Domain Prediction from Max F1,cdf_area Domain Prediction from Max F1
0,Ag1Al19.8Co19.8Cu4.95Zr54.45,12.722,6.907153,0.359390,ID,ID
1,Ag1Al9.9Cu22.77Fe4.95Zr61.38,6.048,5.429492,0.391617,ID,ID
2,Ag1Cu5P5Pd79Si10,4.704,4.398859,1.000000,OD,OD


In [17]:
# Predict the Rc (log scale) of metallic glasses

comps_list = ['Zr66Al8Ni26', 'Zr66Al8Cu7Ni19', 'Zr66Al8Cu12Ni14']
datatype_list = ['expt', 'expt', 'expt']
result = garden.predict_metallicglass_Rc(comps_list, datatype_list)

result

Output()

(worker SDK version: 2.16.0; worker OS: Linux-3.10.0-1160.90.1.el7.x86_64-x86_64-with-glibc2.36)


,Predicted log Rc (K/s),Ebar log Rc (K/s),d_pred,rmse/std_y Domain Prediction from Max F1,cdf_area Domain Prediction from Max F1,quality_4,quality_3,Row_composition_average,Period_composition_average,Electronegativity_difference,...,ElectricalConductivity_difference,ThermalExpansionCoefficient_difference,BCCfermi_composition_average,NpValence_composition_average,MendeleevNumber_arithmetic_average,AtomicNumber_composition_average,MeltingT_difference,MendeleevNumber_difference,Group_difference,BCCvolume_padiff_composition_average
0,1.885113,0.382386,0.573878,ID,OD,0,0,4.58,4.58,0.58,...,35.4,17.4,4.736545,0.08,59.333333,34.72,1194.53,29.0,9.0,-0.2559
1,1.225258,0.331150,0.744672,ID,OD,0,0,4.58,4.58,0.58,...,58.4,17.4,4.686352,0.08,60.500000,34.79,1194.53,29.0,9.0,-0.2559
2,1.028105,0.293351,0.745410,ID,OD,0,0,4.58,4.58,0.58,...,58.4,17.4,4.650499,0.08,60.500000,34.84,1194.53,29.0,9.0,-0.2559


In [18]:
# Predict the Rc (log scale) of metallic glasses, data from LLMs

comps_list = ['Cu30Ni10P20Pd40', 'Cu0.5Zr0.5','Ni40P20Pd40']
result = garden.predict_metallicglass_Rc_LLM(comps_list)

result

Output()

(worker SDK version: 2.16.0; worker OS: Linux-3.10.0-1160.92.1.el7.x86_64-x86_64-with-glibc2.36)


,Compositions,Predicted log Rc (K/s),Ebar log Rc (K/s),d_pred,rmse/std_y Domain Prediction from Max F1,cdf_area Domain Prediction from Max F1
0,Cu30Ni10P20Pd40,-0.967101,0.851929,0.755110,ID,OD
1,Cu0.5Zr0.5,1.380936,1.485606,0.440647,ID,OD
2,Ni40P20Pd40,-0.453406,0.920585,0.919078,ID,OD


In [19]:
# Predict Mg alloy yield strength
import pandas as pd
d_test = {'Mg(wt.%)' : [99.27, 98.77],
          'Zn(wt.%)': [0, 0],
          'Y(wt.%)': [0.73, 1.23],
          'Zr(wt.%)': [0, 0],
          'Nd(wt.%)': [0, 0],
          'Gd(wt.%)': [0, 0],
          'solution temperature(°Ê)': [525, 525],
          'solution time(h)': [7, 7],
          'homogenization temperature(°Ê)': [0, 0],
          'homogenization time(h)': [0, 0],
          'extrusion temperature(°Ê)': [0, 0],
          'extrusion ratio': [0, 0],
          'aging temperature(°Ê)': [0, 0],
          'aging time(h)': [0, 0]}
result = garden.predict_Mg_alloy(pd.DataFrame(d_test))
result

Output()

(worker SDK version: 2.16.0; worker OS: Linux-3.10.0-1160.92.1.el7.x86_64-x86_64-with-glibc2.36)


,Predicted Mg alloy yield strength (MPa),Ebar Mg alloy yield strength (MPa),d_pred,rmse/std_y Domain Prediction from Max F1,cdf_area Domain Prediction from Max F1,extrusion ratio,Mg(wt.%),extrusion temperature(°Ê),homogenization time(h),Y(wt.%),Zn(wt.%),aging time(h)
0,36.420,31.461345,0.367611,ID,OD,0,99.27,0,0,0.73,0,0
1,52.072,34.830584,0.338591,ID,OD,0,98.77,0,0,1.23,0,0


In [20]:
# Predict the thermal conductivity (log scale) of materials

comps_list = ['BeS', 'CdS', 'GaN']
temps_list = [300, 300, 300]
result = garden.predict_thermal_conductivity(comps_list, temps_list)

result

Output()

(worker SDK version: 2.16.0; worker OS: Linux-3.10.0-1160.92.1.el7.x86_64-x86_64-with-glibc2.36)


,Compositions,Predicted log thermal conductivity (W/m-K),Ebar log thermal conductivity (W/m-K),d_pred,rmse/std_y Domain Prediction from Max F1,cdf_area Domain Prediction from Max F1
0,BeS,2.183949,0.281531,0.994828,ID,OD
1,CdS,1.299493,0.131778,0.950717,ID,OD
2,GaN,2.271398,0.210169,1.000000,ID,OD


In [21]:
# Predict the superconducting critical temperature (natural log scale) of materials

comp_list = ['Ba0.4K0.6Fe2As2', 'Ca0.4Ba1.25La1.25Cu3O6.98', 'La1.71Sr0.29Cu0.94Co0.06O4']
result = garden.predict_superconductivity(comp_list)

result

Output()

(worker SDK version: 2.16.0; worker OS: Linux-3.10.0-1160.92.1.el7.x86_64-x86_64-with-glibc2.36)


,Compositions,Predicted ln Tc (K),Ebar ln Tc (K),d_pred,rmse/std_y Domain Prediction from Max F1,cdf_area Domain Prediction from Max F1
0,Ba0.4K0.6Fe2As2,3.344594,0.173041,1.000000,ID,OD
1,Ca0.4Ba1.25La1.25Cu3O6.98,3.890828,0.275220,0.575955,ID,ID
2,La1.71Sr0.29Cu0.94Co0.06O4,3.487506,0.096015,0.773313,ID,ID


In [22]:
# Predict the yield strength of steel alloys

comp_list = ['Fe0.620C0.000953Mn0.000521Si0.00102Cr0.000110Ni0.192Mo0.0176V0.000112Nb0.0000616Co0.146Al0.00318Ti0.0185',
                     'Fe0.623C0.00854Mn0.000104Si0.000203Cr0.147Ni0.0000971Mo0.0179V0.00515N0.00163Nb0.0000614Co0.188W0.00729Al0.000845',
                     'Fe0.625Mn0.000102Si0.000200Cr0.0936Ni0.129Mo0.00480V0.000110Nb0.0000604Co0.132Al0.00812Ti0.00669']
result = garden.predict_steel_yield(comp_list)

result

Output()

(worker SDK version: 2.16.0; worker OS: Linux-3.10.0-1160.90.1.el7.x86_64-x86_64-with-glibc2.36)


,Compositions,Predicted steel yield strength (MPa),Ebar steel yield strength (MPa),d_pred,rmse/std_y Domain Prediction from Max F1,cdf_area Domain Prediction from Max F1
0,Fe0.620C0.000953Mn0.000521Si0.00102Cr0.000110N...,2388.195333,90.866087,1.000000,OD,OD
1,Fe0.623C0.00854Mn0.000104Si0.000203Cr0.147Ni0....,1180.790000,128.677058,0.722452,ID,ID
2,Fe0.625Mn0.000102Si0.000200Cr0.0936Ni0.129Mo0....,1697.142000,144.799390,0.975964,OD,OD


In [ ]:
# Predict the charge state transition levels of defects in semiconductors

comp_list = ['CdSV', 'CdSIn'] # V and In are impurities in host CdS
chargefrom_list = [2, 2]
chargeto_list = [1, 1]
lattice_list = [5.94, 5.94]
bandgap_list = [2.05, 2.05]
is_b_latt_list = [0, 0]
result = garden.predict_semiconductor_lvls(comp_list, chargefrom_list, chargeto_list, lattice_list, bandgap_list, is_b_latt_list)

result

Output()

In [ ]:
# Predict the area specific resistance (ASR) of perovskite oxygen catalyst materials

comp_list = ['Ba0.9Co0.7Fe0.2Nb0.1O3', 'Ba2Bi0.1Sc0.2Co1.7O5', 'Ba0.95Ca0.05FeO3']
elec_list = ['ceria', 'mixed', 'ceria']
result = garden.predict_perovskite_ASR(comp_list, elec_list)

result

In [ ]:
# Predict the Hydrogen uptake of perovskite oxygen catalyst materials

comp_list = ['La0.7Sr0.3MnO2.95', 'La0.7Sr0.3MnO2.95']
temp_list = [415, 415]
ph2o_list = [0.023, 0.09]
result = garden.predict_perovskite_Habs(comp_list, temp_list, ph2o_list)

result

In [ ]:
# Predict the O 2p-band center of perovskite oxides

comp_list = ['Ba0.25Ca0.75MoO3', 'Ba0.25Ca0.75VO3', 'Ba0.25Ca0.75ZrO3']
result = garden.predict_perovskite_Opband(comp_list)

result

In [ ]:
# Predict the thermal expansion coefficient of perovskites

comp_list = ['La0.6Sr0.4Fe0.8Cu0.2O3', 'La0.8Sr0.2Mn0.75Co0.25O3', 'La0.8Sr0.2Mn0.5Co0.5O3']
result = garden.predict_perovskite_tec(comp_list)

result

In [ ]:
# Predict the work function of AO-termination (001) surface of perovskites

comp_list = ['RbAlO3', 'RbGaO3', 'RbInO3']
result = garden.predict_perovskite_workfunction(comp_list)

result

In [ ]:
# Predict the maximum phonon frequency of materials

comp_list = ['BaTe', 'AsB', 'BaGaHSi']
result = garden.predict_phonon_freq(comp_list)

result

In [ ]:
# Predict the Debye temperature of materials

comp_list = ['Zr1', 'K2Mg5Sn3', 'CInLa3']
result = garden.predict_debyeT_aflow(comp_list)

result

In [ ]:
# Predict the thermal expansion coefficient of materials

comp_list = ['Ho2InNi2', 'LaSe', 'MnSiTb']
result = garden.predict_thermalexp_aflow(comp_list)

result

In [ ]:
# Predict the thermal conductivity of materials

comp_list = ['Ho2InNi2', 'LaSe', 'MnSiTb']
result = garden.predict_thermalcond_aflow(comp_list)

result

In [ ]:
# Predict the total conductivity of perovskite materials

comp_list = ['(BaO)0.1(SrO)0.9(CeO2)0.5(ZrO2)0.35(Y2O3)0.05(Sm2O3)0.025', '(BaO)0.1(SrO)0.9(CeO2)0.5(ZrO2)0.35(Y2O3)0.05(Sm2O3)0.025']
temp_list = [700.716623, 649.022285]
ph2o_list = [1.9, 1.9]
po2_list = [0, 0]
result = garden.predict_perovskite_conductivity(comp_list, temp_list, ph2o_list, po2_list)

result

In [ ]:
# Predict the formation energy of perovskite materials

comp_list = ['Bi1F1Hf1O2', 'F1Li1N1O1Y1', 'As1N1O2Sc1']
result = garden.predict_perovskite_formationE(comp_list)

result

In [ ]:
# Predict the stability (convex hull energy) of perovskite oxides

comp_list = ['Ba0.875Zn0.125CoO3', 'Ba0.875Zn0.125FeO3', 'Ba0.875Zn0.125MnO3']
result = garden.predict_perovskite_stability(comp_list)

result

In [ ]:
# Predict the ductile-to-brittle transition temperature shift of reactor pressure vessel steel alloys

d_test = {'temperature_C' : [287, 287],
          'wt_percent_Cu': [0.04, 0.04],
          'wt_percent_Ni': [0.74, 0.74],
          'wt_percent_Mn': [0.6, 0.6],
          'wt_percent_P': [0.012, 0.012],
          'wt_percent_Si': [0.23, 0.23],
          'wt_percent_C': [0.21, 0.21],
          'log(fluence_n_cm2)': [18.6645479622466, 19.6419695977021],
          'log(flux_n_cm2_sec)': [11.1657413553112, 11.1608917577271],
          'Product Form': ["F", "F"]}

result = garden.predict_RPV_TTS(d_test)

result

In [ ]:
# Predict oxygen vacancy formation energy in oxides

comp_list = ['Ac2O3', 'AcAgO3', 'AcAlO3']

result = garden.predict_oxide_vacancy(comp_list)

result